In [3]:
import numpy as np
import pandas as pd

# Input data files are available in the read-only "../input/" directory|
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic-competition/train.csv
/kaggle/input/titanic-competition/test.csv
/kaggle/input/titanic-competition/gender_submission.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [12]:
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
train = pd.read_csv('/kaggle/input/titanic/train.csv')

In [13]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if substring in big_string:
            return substring
    print(big_string)
    return np.nan
    return 'Unknown'

title_list = ['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
              'Dr', 'Ms', 'Mlle', 'Col', 'Capt', 'Mme', 'Countess',
              'Don', 'Jonkheer']

def extract_title(name):
    for title in title_list:
        if title in name:
            return title
    return 'Other'

# Example usage:
names = ['Mrs. Smith', 'Mr. Johnson', 'Miss. Davis', 'Dr. Brown']

for name in names:
    title = extract_title(name)
    print(f"Name: {name}, Title: {title}")

Name: Mrs. Smith, Title: Mrs
Name: Mr. Johnson, Title: Mr
Name: Miss. Davis, Title: Miss
Name: Dr. Brown, Title: Dr


# I recombine them into four categories

In [14]:
train['Title'] = train['Name'].map(lambda x: substrings_in_string(x, title_list))

# replacing all titles with 'Mr', 'Mrs', 'Miss', 'Master'
def replace_titles(x):
    title = x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title == 'Dr':
        if x['Sex'] == 'Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

train['Title'] = train.apply(replace_titles, axis=1)

In [15]:
def substrings_in_string(big_string, substrings):
    if pd.isna(big_string):
        return 'Unknown'
    
    for substring in substrings:
        if substring in big_string:
            return substring
    return 'Unknown'

cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
test['Deck'] = test['Cabin'].map(lambda x: substrings_in_string(str(x), cabin_list))


# Exploring the datasets:

In [16]:
gender_submission.head(1)

,PassengerId,Survived
0,892,0


In [17]:
test.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Unknown


In [18]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [19]:
from sklearn.impute import SimpleImputer

# Display columns with missing values and their counts
missing_values = train.isnull().sum()
print("Columns with missing values:")
print(missing_values[missing_values > 0])

Columns with missing values:
Age         177
Cabin       687
Embarked      2
dtype: int64


In [20]:
# Impute missing values for specific columns
imputer = SimpleImputer(strategy='most_frequent')  # You can use different strategies (mean, median, constant) based on the data
columns_to_impute = ['Age', 'Embarked']  # Columns to impute missing values
train[columns_to_impute] = imputer.fit_transform(train[columns_to_impute])

# Check if missing values have been filled
missing_values_after_imputation = train[columns_to_impute].isnull().sum()
print("\nMissing values after imputation:")
print(missing_values_after_imputation)


Missing values after imputation:
Age         0
Embarked    0
dtype: int64


In [21]:
# Perform one-hot encoding for 'Sex', 'Embarked', and 'Title' columns
train = pd.get_dummies(train, columns=['Sex', 'Embarked', 'Title'])

# Display the updated DataFrame with one-hot encoded columns
print(train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0      1      0   
2                             Heikkinen, Miss. Laina  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0      1      0   
4                           Allen, Mr. William Henry  35.0      0      0   

             Ticket     Fare Cabin  Sex_female  Sex_male  Embarked_C  \
0         A/5 21171   7.2500   NaN       False      True       False   
1          PC 17599  71.2833   C85        True     False        True   
2  STON/O2. 3101282   7.9250   NaN        True     False       False   
3            113803  53.1000  C123    

Creating a new feature to represent the total family size by combining 'SibSp' and 'Parch' is a useful way to analyze the impact of family size on survival rates.

In [22]:
# Create a new feature 'Family_Size' by adding 'SibSp' and 'Parch' columns
train['Family_Size'] = train['SibSp'] + train['Parch']

# Display the updated DataFrame with the new 'Family_Size' feature
print(train[['SibSp', 'Parch', 'Family_Size']].head())

   SibSp  Parch  Family_Size
0      1      0            1
1      1      0            1
2      0      0            0
3      1      0            1
4      0      0            0


2. Grouping rare titles or titles based on their social status can indeed be a beneficial feature engineering step.

In [23]:
# Extract titles from the 'Name' column
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Display unique titles and their counts
print(train['Title'].value_counts())

# Map titles to specific categories based on their rarity or social status
title_mapping = {
    'Mr': 'Mr',
    'Miss': 'Miss',
    'Mrs': 'Mrs',
    'Master': 'Master',
    'Dr': 'Rare',
    'Rev': 'Rare',
    'Col': 'Rare',
    'Major': 'Rare',
    'Mlle': 'Rare',
    'Countess': 'Rare',
    'Ms': 'Rare',
    'Lady': 'Rare',
    'Jonkheer': 'Rare',
    'Don': 'Rare',
    'Dona': 'Rare',
    'Mme': 'Rare',
    'Capt': 'Rare',
    'Sir': 'Rare'
}

# Map the titles to their respective categories
train['Title'] = train['Title'].map(title_mapping)

# Display the updated DataFrame with the new 'Title' categories
print(train['Title'].value_counts())


Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: count, dtype: int64
Title
Mr        517
Miss      182
Mrs       125
Master     40
Rare       27
Name: count, dtype: int64


3. Extracting deck information from the 'Cabin' column and creating a new feature based on the deck letters can be done by parsing the cabin information and extracting the deck letters.

In [24]:
# Extracting the deck information from the 'Cabin' column
train['Deck'] = train['Cabin'].str.slice(0, 1)  # Extract the first character of the 'Cabin' column

# Display unique deck values and their counts
print(train['Deck'].value_counts())

# Fill missing values with a specific category (if needed)
train['Deck'].fillna('Unknown', inplace=True)  # Fill missing values with 'Unknown' category

# Display the updated DataFrame with the new 'Deck' column
print(train[['Cabin', 'Deck']].head())

Deck
C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: count, dtype: int64
  Cabin     Deck
0   NaN  Unknown
1   C85        C
2   NaN  Unknown
3  C123        C
4   NaN  Unknown


In [25]:
# Extracting prefixes from the 'Ticket' column
train['Ticket_Prefix'] = train['Ticket'].apply(lambda x: x.split(' ')[0] if len(x.split(' ')) > 1 else 'NoPrefix')

# Display unique ticket prefixes and their counts
print(train['Ticket_Prefix'].value_counts())

# Display the updated DataFrame with the new 'Ticket_Prefix' column
print(train[['Ticket', 'Ticket_Prefix']].head())


Ticket_Prefix
NoPrefix      665
PC             60
C.A.           27
STON/O         12
A/5            10
W./C.           9
CA.             8
SOTON/O.Q.      8
SOTON/OQ        7
A/5.            7
CA              6
STON/O2.        6
C               5
F.C.C.          5
S.O.C.          5
SC/PARIS        5
SC/Paris        4
S.O./P.P.       3
PP              3
A/4.            3
A/4             3
SC/AH           3
A./5.           2
SOTON/O2        2
A.5.            2
WE/P            2
S.C./PARIS      2
P/PP            2
F.C.            1
SC              1
S.W./PP         1
A/S             1
Fa              1
SCO/W           1
SW/PP           1
W/C             1
S.C./A.4.       1
S.O.P.          1
A4.             1
W.E.P.          1
SO/C            1
S.P.            1
C.A./SOTON      1
Name: count, dtype: int64
             Ticket Ticket_Prefix
0         A/5 21171           A/5
1          PC 17599            PC
2  STON/O2. 3101282      STON/O2.
3            113803      NoPrefix
4            373

4. Convert the 'Age' feature into categorical bins representing different age groups using pandas' cut() function. This enables you to create age categories such as 'child', 'adult', 'elderly', etc., and analyze potential survival patterns among these groups

In [26]:
# Define the age bin edges and labels for different age groups
age_bins = [0, 12, 18, 60, float('inf')]  # Define bin edges for 'child', 'teen', 'adult', and 'elderly'
age_labels = ['Child', 'Teen', 'Adult', 'Elderly']  # Labels for different age groups

# Create a new column 'Age_Group' containing the categorical age bins
train['Age_Group'] = pd.cut(train['Age'], bins=age_bins, labels=age_labels, right=False)

# Display unique values and counts in the 'Age_Group' column
print(train['Age_Group'].value_counts(dropna=False))

# Display the updated DataFrame with the new 'Age_Group' column
print(train[['Age', 'Age_Group']].head())


Age_Group
Adult      752
Child       68
Teen        45
Elderly     26
Name: count, dtype: int64
    Age Age_Group
0  22.0     Adult
1  38.0     Adult
2  26.0     Adult
3  35.0     Adult
4  35.0     Adult


5. Calculate the fare per person by dividing the 'Fare' by the 'Family Size' (including the individual), providing insights into survival based on ticket prices paid per person.

In [27]:
# Calculate fare per person by dividing 'Fare' by 'Family_Size'
train['Fare_Per_Person'] = train['Fare'] / (train['Family_Size'] + 1)

# Display the updated DataFrame with the new 'Fare_Per_Person' column
print(train[['Fare', 'Family_Size', 'Fare_Per_Person']].head())

      Fare  Family_Size  Fare_Per_Person
0   7.2500            1          3.62500
1  71.2833            1         35.64165
2   7.9250            0          7.92500
3  53.1000            1         26.55000
4   8.0500            0          8.05000


6. Create a binary feature indicating missing values in a particular column (e.g., 'Cabin_missing', 'Age_missing') to leverage the information of missingness.

In [28]:
# Create binary indicators for missing values in 'Cabin' and 'Age' columns
train['Cabin_missing'] = train['Cabin'].isnull().astype(int)
train['Age_missing'] = train['Age'].isnull().astype(int)

# Display the updated DataFrame with binary indicators for missing values
print(train[['Cabin', 'Cabin_missing', 'Age', 'Age_missing']].head())

  Cabin  Cabin_missing   Age  Age_missing
0   NaN              1  22.0            0
1   C85              0  38.0            0
2   NaN              1  26.0            0
3  C123              0  35.0            0
4   NaN              1  35.0            0


# Model Building

In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   PassengerId      891 non-null    int64   
 1   Survived         891 non-null    int64   
 2   Pclass           891 non-null    int64   
 3   Name             891 non-null    object  
 4   Age              891 non-null    object  
 5   SibSp            891 non-null    int64   
 6   Parch            891 non-null    int64   
 7   Ticket           891 non-null    object  
 8   Fare             891 non-null    float64 
 9   Cabin            204 non-null    object  
 10  Sex_female       891 non-null    bool    
 11  Sex_male         891 non-null    bool    
 12  Embarked_C       891 non-null    bool    
 13  Embarked_Q       891 non-null    bool    
 14  Embarked_S       891 non-null    bool    
 15  Title_Master     891 non-null    bool    
 16  Title_Miss       891 non-null    bool    
 1

1. The process of dividing your dataset into training, validation, and test sets typically involves allocating a certain percentage of your data to each set.

In [31]:
from sklearn.model_selection import train_test_split

X = train[['Title_Mr', 'Title_Mrs', 'Title_Master', 'Title_Miss', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Family_Size','Fare_Per_Person', 'Sex_male', 'Sex_female', 'Cabin_missing', 'Age_missing']]  # Features
y = train['Survived']  # Target variable

# Assuming your features are stored in X and the target variable in y
# X contains your input features, and y contains the corresponding labels or outputs

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets (70% training, 30% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Now you have:
# X_train, y_train -> Training set
# X_val, y_val     -> Validation set
# X_test, y_test   -> Test set

# Ensure to replace 'X' and 'y' with your actual feature and target variable names respectively.
# The 'random_state' parameter ensures reproducibility by fixing the random seed.

# Model Selection, Training and Evaluation

In [33]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
logistic_model = LogisticRegression(random_state=42)

# Train the model on the training data
logistic_model.fit(X_train, y_train)

# Evaluate the model on the validation set
validation_accuracy = logistic_model.score(X_val, y_val)
print(f"Validation Set Accuracy (Logistic Regression): {validation_accuracy:.4f}")

# Evaluate the final performance on the test set
test_accuracy = logistic_model.score(X_test, y_test)
print(f"Test Set Accuracy (Logistic Regression): {test_accuracy:.4f}")

Validation Set Accuracy (Logistic Regression): 0.8271
Test Set Accuracy (Logistic Regression): 0.7989


In [34]:
from sklearn.svm import SVC

# Initialize the Support Vector Machine (SVM) model
svm_model = SVC(random_state=42)

# Train the SVM model on the training data
svm_model.fit(X_train, y_train)

# Evaluate the model on the validation set
validation_accuracy = svm_model.score(X_val, y_val)
print(f"Validation Set Accuracy (SVM): {validation_accuracy:.4f}")

# Evaluate the final performance on the test set
test_accuracy = svm_model.score(X_test, y_test)
print(f"Test Set Accuracy (SVM): {test_accuracy:.4f}")

Validation Set Accuracy (SVM): 0.6402
Test Set Accuracy (SVM): 0.6760


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize the Neural Network model
model = Sequential()

# Add input and hidden layers
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization

# Add output layer
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training data
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Set Accuracy (Neural Network): {test_accuracy:.4f}")

In [36]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the Naive Bayes model (Multinomial Naive Bayes)
naive_bayes_model = MultinomialNB()

# Train the model on the training data
naive_bayes_model.fit(X_train, y_train)

# Evaluate the model on the validation set
validation_accuracy = naive_bayes_model.score(X_val, y_val)
print(f"Validation Set Accuracy (Naive Bayes): {validation_accuracy:.4f}")

# Evaluate the final performance on the test set
test_accuracy = naive_bayes_model.score(X_test, y_test)
print(f"Test Set Accuracy (Naive Bayes): {test_accuracy:.4f}")

Validation Set Accuracy (Naive Bayes): 0.7523
Test Set Accuracy (Naive Bayes): 0.7486


In [37]:
import xgboost as xgb

# Initialize the XGBoost Classifier model
xgb_model = xgb.XGBClassifier(random_state=42)

# Train the model on the training data
xgb_model.fit(X_train, y_train)

# Evaluate the model on the validation set
validation_accuracy = xgb_model.score(X_val, y_val)
print(f"Validation Set Accuracy (XGBoost): {validation_accuracy:.4f}")

# Evaluate the final performance on the test set
test_accuracy = xgb_model.score(X_test, y_test)
print(f"Test Set Accuracy (XGBoost): {test_accuracy:.4f}")


Validation Set Accuracy (XGBoost): 0.7944
Test Set Accuracy (XGBoost): 0.8380
